In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
# import requests

In [ ]:

candidate_cells = ['Wien', 'Thessaloniki', 'Batumi', 'Limassol', 'Ganja'
, 'Capital City of Budapest', 'Kharkiv (Харків)', 'Luzern', 'Korzeniew', 'Almaty', 'Kaunas', 'Haraçinë', 'Poitiers', 'Tallinn',
 'Ljubljana', 'Երևան', 'Pristina', 'Tuzla', 'Bergen', 'Budapest főváros', 'Istanbul', 'Cwmffrwd', 'Yerevan']

def dbo_sparql_results(query_string):
    sparql = SPARQLWrapper('https://dbpedia.org/sparql')
    sparql.setQuery(query_string)
    
    try:
        sparql.setReturnFormat(JSON)
        qres = sparql.query().convert()
        return qres
    except:
        pass


def get_dbo_classes_sparql(candidate_cells):
    
    classes = list([])

    dbo_prefix = 'http://dbpedia.org/ontology/'
    for cell in candidate_cells:
        print(f'###################{cell}########################')
        query_string = f'''
        SELECT ?class
        WHERE {{ dbr:{cell} a ?class.
        }}'''
    
#         query_string = f'''
#         select distinct ?superclass 
#         where {{dbr:{cell} rdf:type ?e. 
#             ?e rdfs:subClassOf* ?superclass.
#         FILTER (strstarts(str(?superclass), '{dbo_prefix}'))}}'''
        
#         print(query_string)
    
        qres = dbo_sparql_results(query_string)
#         pprint(qres)
        try:
            for entity_class in qres['results']['bindings']:
                if dbo_prefix in entity_class[list(qres['results']['bindings'][1].keys())[0]]['value']:
                    candicate_class = entity_class[list(qres['results']['bindings'][1].keys())[0]]['value'].split(dbo_prefix)[1]
                    classes.append(candicate_class)
                    print(candicate_class)
        except:
            pass
    return set(classes)

classes = get_dbo_classes_sparql(candidate_cells)
classes

In [2]:
def get_dbo_class_entities_sparql(candidate_classes):
    sparql = SPARQLWrapper('https://dbpedia.org/sparql')
    
    ent_dict = dict()

    dbp_prefix = 'http://dbpedia.org/resource/'
    for candidate_class in candidate_classes:
        print(f'###################{candidate_class}########################')
        sparql.setQuery(f'''
        SELECT ?object
        WHERE {{ ?object a dbo:{candidate_class}. }}
        ORDER BY RAND()
        LIMIT 10
        OFFSET 10000
        # OFFSET """ + str(i) + "0000"
        ''')
        ent_dict[candidate_class] = list([])
        try:
            sparql.setReturnFormat(JSON)
            qres = sparql.query().convert()
#             pprint(qres)
            for entity in qres['results']['bindings']:
#                 print(entity['object']['value'].split(dbp_prefix)[1])
                ent_dict[candidate_class].append(entity['object']['value'].split(dbp_prefix)[1])
#                 if dbo_prefix in entity_class['class']['value']:
#                     candicate_class = entity_class['class']['value'].split(dbo_prefix)[1]
#                     classes.append(candicate_class)
#                     print(candicate_class)
        except:
            pass
        
    return ent_dict

general_entities = get_dbo_class_entities_sparql(['GovernmentAgency', 'Eukaryote', 'PeriodicalLiterature'])

###################GovernmentAgency########################
###################Eukaryote########################
###################PeriodicalLiterature########################


In [21]:
# general_entities

In [3]:
general_entities

{'GovernmentAgency': ['Ministry_of_Armed_Forces_(France)',
  'Independent_Communications_Authority_of_South_Africa',
  'Tourism_Ireland',
  'Department_of_Energy_and_Resource_Development_(New_Brunswick)',
  'Rock_the_World_Youth_Mission_Alliance',
  'Department_of_Environment,_Forestry_and_Fisheries',
  'Ministry_of_Agriculture_and_Forestry_(Finland)',
  'National_Library_Service_of_Malawi',
  'Ministry_of_National_Competitiveness_and_the_fight_against_the_High_Cost_of_Life_(Niger)',
  'Regional_Transport_Office'],
 'Eukaryote': ['Cereza',
  'Tarzino',
  'Hamaxia_mutatum',
  'Phylloxiphia_oweni',
  'Charltona_kala',
  'Platamonina',
  'Tonna_deshayesii',
  'Aeolostoma',
  'Bradburia_pilosa',
  'Striped_treeshrew'],
 'PeriodicalLiterature': ['Numerical_Methods_for_Partial_Differential_Equations',
  'Meggendorfer-Blätter',
  'King_County_Journal',
  'American_Sociological_Review',
  'Khurzarin',
  'La_Discusión',
  'Rambam_Maimonides_Medical_Journal',
  'Standpoint_(magazine)',
  'La_Hum

In [6]:
# Query DBPedia for an entity's complete classes
def query_complete_classes_of_entity(ent):
    classes = set()
    try:
        s = sparql.Service('http://dbpedia.org/sparql', "utf-8", "GET")
        statement = 'select distinct ?superclass where { <%s%s> rdf:type ?e. ' \
            '?e rdfs:subClassOf* ?superclass. FILTER(strstarts(str(?superclass), "%s"))}' \
            % (dbp_prefix, ent, dbo_prefix)
        result = s.query(statement)
        for row in result.fetchone():
            cls_uri = str(row[0])
            cls = cls_uri.split(dbo_prefix)[1]
            classes.add(cls)
    except UnicodeDecodeError:
        print('     %s: UnicodeDecodeError' % ent)
        pass
    return classes

temp = query_complete_classes_of_entity('Athens')

NameError: name 'sparql' is not defined

In [ ]:
# TO REWRITE

def super_classes(col_classes):
    dbo_prefix = 'http://dbpedia.org/ontology/'
    for i, col in enumerate(col_classes.keys()):
        ori_cls = col_classes[col][0]
        statement = 'SELECT distinct ?superclass WHERE { dbo:%s rdfs:subClassOf* ?superclass. ' \
                    'FILTER ( strstarts(str(?superclass), "%s"))}' % (ori_cls, dbo_prefix)
        result = dbo_sparql_results(statement)
        for row in result:
            super_cls = str(row[0])
            super_cls_name = super_cls.split(dbo_prefix)[1]
            if super_cls_name not in col_classes[col]:
                col_classes[col].append(super_cls_name)
        if i % 10 == 0:
            print('%d columns done' % (i + 1))
    return col_classes